# CSV Generator

## Intención de este notebook

Generar el CSV con la información final de los vuelos, que será utilizando en Neo4j para resolver las consultas propuestas por la cátedra.

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians

## Procesamiento de los datos

### Carga del archivo pkl

In [10]:
info_vuelos = pd.read_pickle('pkl/info_vuelos.pkl')
info_vuelos = info_vuelos.drop(columns=['vuelo', 'terminal_salida', 'puerta_salida', 'terminal_llegada', 'puerta_llegada'])
info_vuelos.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
37805,Air France,Boston,Atlanta,"[martes, 12, marzo]",11:10,"[martes, 12, marzo]",13:05
97467,China Southern,Hangzhou,Dalián,"[viernes, 15, marzo]",06:40,"[viernes, 15, marzo]",09:48
76254,Skymark,Sapporo New Chitose,Tokio Haneda,"[martes, 12, marzo]",19:08,"[martes, 12, marzo]",08:25
43353,Air France,Viena,París Charles de Gaulle,"[martes, 12, marzo]",18:40,"[martes, 12, marzo]",20:45
172258,easyJet,Barcelona,Berlín Schönefeld,"[jueves, 21, marzo]",13:20,"[jueves, 21, marzo]",15:45


In [11]:
info_vuelos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 344124 entries, 0 to 196239
Data columns (total 7 columns):
aerolinea          344124 non-null object
origen             344124 non-null object
destino            344124 non-null object
fecha_salida       344124 non-null object
horario_salida     344124 non-null object
fecha_llegada      344124 non-null object
horario_llegada    344124 non-null object
dtypes: object(7)
memory usage: 21.0+ MB


### Eliminación de espacios

In [12]:
info_vuelos['aerolinea'] = info_vuelos['aerolinea'].apply(lambda x: x.strip())
info_vuelos['origen'] = info_vuelos['origen'].apply(lambda x: x.strip())
info_vuelos['destino'] = info_vuelos['destino'].apply(lambda x: x.strip())
info_vuelos.sample(5)

,aerolinea,origen,destino,fecha_salida,horario_salida,fecha_llegada,horario_llegada
36388,Air France,Atlanta,Des Moines,"[miercoles, 13, marzo]",01:24,"[miercoles, 13, marzo]",08:44
90747,COPA,Chicago O'Hare,Panamá,"[viernes, 15, marzo]",09:55,"[viernes, 15, marzo]",11:50
89056,Airlines PNG,Puerto Moresby,Alotau,"[martes, 12, marzo]",09:55,"[martes, 12, marzo]",10:59
75767,British Airways,Stuttgart,Londres Heathrow,"[lunes, 11, marzo]",06:47,"[lunes, 11, marzo]",08:35
101231,China Southern,Shijiazhuang,Hangzhou,"[martes, 12, marzo]",13:50,"[martes, 12, marzo]",16:25


### Conversión de fechas y horarios

In [17]:
# Obtenemos el numero del mes
def get_month_number(month):
    months = {'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06',\
              'julio': '07', 'agosto': '08', 'septiembre': '09]', 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    return months[month]

# Obtenemos la fecha en formato timestamp (YYYY-MM-DDTHH:MM)
def get_timestamp(date, hour):
    day = str(date[1])
    if len(day) == 1:
        day = '0' + day
    month = get_month_number(date[2])
    timestamp = '2019' + '-' + month + '-' + day + 'T' + hour
    
    return timestamp

# Cargamos los timestamps al dataframe
info_vuelos['hora_salida'] = info_vuelos.apply(lambda row: get_timestamp(row.fecha_salida, row.horario_salida), axis=1)
info_vuelos['hora_llegada'] = info_vuelos.apply(lambda row: get_timestamp(row.fecha_llegada, row.horario_llegada), axis=1)

# Eliminamos columnas innecesarias
info_vuelos = info_vuelos.drop(columns=['fecha_salida', 'horario_salida', 'fecha_llegada', 'horario_llegada'])

info_vuelos.sample(5)

,aerolinea,origen,destino,hora_salida,hora_llegada
2818,American,Charlotte,Baltimore/Washington,2019-03-08T15:48,2019-03-08T17:11
12437,American,Nueva York JFK,Raleigh,2019-03-10T12:10,2019-03-10T13:53
79403,EVA Air,Taiwán Taoyuan,Bangkok Suvarnabhumi,2019-03-12T09:10,2019-03-12T12:07
81077,Air China,Bangkok Suvarnabhumi,Pekín-Capital,2019-03-13T08:05,2019-03-14T10:48
59306,Avianca,Bogotá,Barrancabermeja,2019-03-11T09:38,2019-03-11T11:06


### Duración del vuelo (en minutos)

In [38]:
def get_duration(departure_date, arrival_date):
    t1 = pd.Timestamp(departure_date)
    t2 = pd.Timestamp(arrival_date)
    
    return int(pd.Timedelta(t2 - t1).total_seconds() / 60)

info_vuelos['duracion (min)'] = info_vuelos.apply(lambda row: get_duration(row.hora_salida, row.hora_llegada), axis=1)

info_vuelos.sample(5)

,aerolinea,origen,destino,hora_salida,hora_llegada,duracion (min)
126615,Delta,Sint Maarten,Atlanta,2019-03-12T20:25,2019-03-12T22:40,135
90768,COPA,Pereira,Panamá,2019-03-15T22:31,2019-03-15T23:21,50
6962,American,Dallas/Fort Worth,Brownsville,2019-03-09T07:55,2019-03-09T09:15,80
3388,American,Charlotte,Fráncfort del Meno,2019-03-07T12:46,2019-03-08T14:18,1532
96603,China Southern,Dalián,Seúl Incheon,2019-03-12T07:08,2019-03-12T08:45,97


### Carga del archivo aeropuertos.csv para obtener precio del vuelo

In [40]:
aeropuertos = pd.read_csv('../csv/aeropuertos.csv', index_col=False, names=['codigo', 'aeropuerto', 'capital', 'pais', 'latitud', 'longitud', 'numero1', 'numero2', 'continente'])
aeropuertos.sample(5)

,codigo,aeropuerto,capital,pais,latitud,longitud,numero1,numero2,continente
3395,RET,Røst Airport,Røst,Norway,67.52780,12.10330,7.0,1.00,Europe/Oslo
3061,RUK,Rukumkot Airport,Rukumkot,Nepal,28.62700,82.19500,2500.0,5.75,Asia/Katmandu
3560,RDG,Reading Regional Carl A Spaatz Field,Reading,United States,40.37850,-75.96520,344.0,-5.00,America/New_York
2836,BDJ,Syamsudin Noor Airport,Banjarmasin,Indonesia,-3.44236,114.76300,66.0,8.00,Asia/Makassar
3687,IPA,Ipota Airport,Ipota,Vanuatu,-18.85639,169.28333,23.0,11.00,Pacific/Efate


### Códigos de los aeropuertos origen y destino 

In [41]:
info_aeropuertos = info_vuelos.merge(aeropuertos[['codigo', 'capital']], left_on='origen', right_on='capital', how='left')
info_aeropuertos.dropna(subset=['codigo'], inplace=True)

info_aeropuertos = info_aeropuertos[['aerolinea', 'origen', 'codigo', 'destino', 'hora_salida', 'hora_llegada', 'duracion (min)']]
info_aeropuertos.columns = ['aerolinea', 'origen', 'cod_origen', 'destino','hora_salida', 'hora_llegada', 'duracion (min)']

info_aeropuertos = info_aeropuertos.merge(aeropuertos[['codigo', 'capital']], left_on='destino', right_on='capital', how='left')
info_aeropuertos.dropna(subset=['codigo'], inplace=True)

info_aeropuertos = info_aeropuertos[['aerolinea', 'origen', 'cod_origen', 'destino', 'codigo','hora_salida', 'hora_llegada', 'duracion (min)']]
info_aeropuertos.columns = ['aerolinea', 'origen', 'cod_origen', 'destino', 'cod_destino', 'hora_salida', 'hora_llegada', 'duracion (min)']

In [42]:
info_aeropuertos.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,hora_salida,hora_llegada,duracion (min)
303297,Air France,Tampa,MCF,Atlanta,PDK,2019-03-13T20:14,2019-03-13T21:07,53
51063,American,Charlotte,CLT,Charleston,CHS,2019-03-09T12:20,2019-03-09T13:40,80
361369,Air China,Xiamen,XMN,Guiyang,KWE,2019-03-12T08:24,2019-03-12T09:58,94
333423,Alaska,Seattle,SEA,Anchorage,ANC,2019-03-13T12:55,2019-03-14T16:45,1670
226355,Delta,Cleveland,BKL,Atlanta,PDK,2019-03-15T21:00,2019-03-15T22:00,60


### Funciones para calcular distancia y precio

In [43]:
def get_coordenadas_aeropuerto(cod_aeropuerto):
    info_aeropuerto = aeropuertos[aeropuertos.codigo == cod_aeropuerto].head(1)

    return float(info_aeropuerto.latitud.item()), float(info_aeropuerto.longitud.item())


def calcular_distancia(aeropuerto_origen, aeropuerto_destino):
    aeropuerto_origen_latitud, aeropuerto_origen_longitud = get_coordenadas_aeropuerto(aeropuerto_origen)
    aeropuerto_destino_latitud, aeropuerto_destino_longitud = get_coordenadas_aeropuerto(aeropuerto_destino)
    
    # Approximated radius of earth in km
    R = 6373.0
    
    dlon = aeropuerto_destino_longitud - aeropuerto_origen_longitud
    dlat = aeropuerto_destino_latitud - aeropuerto_origen_latitud
    
    a = sin(dlat / 2)**2 + cos(aeropuerto_origen_latitud) * cos(aeropuerto_destino_latitud) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance


def calcular_precio(distancia_recorrida):
    # Precio por kilometro recorrido para un pasajero
    precio_dolares_por_km = 0.067
    precio = distancia_recorrida * precio_dolares_por_km
    iva = 0.21 * precio
    return precio + iva

### Precio del vuelo (proporcional a la distancia entre los aeropuertos)

In [44]:
info_aeropuertos['precio'] = info_aeropuertos.apply(lambda row: calcular_precio(calcular_distancia(row.cod_origen, row.cod_destino)), axis=1)
info_aeropuertos.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,hora_salida,hora_llegada,duracion (min),precio
368486,China Southern,Wuhan,WUH,Shenzhen,SZX,2019-03-14T13:37,2019-03-14T14:24,47,934.886114
508938,Southwest WN 2085,Santa Ana,SNA,Denver,APA,2019-03-23T20:20,2019-03-23T22:10,110,225.274157
211992,Delta,Atlanta,PDK,Pittsburgh,PIT,2019-03-15T02:02,2019-03-15T05:50,228,927.190331
266901,Air Canada,Newark,EWR,Montreal,YHU,2019-03-08T17:37,2019-03-08T19:34,117,748.449367
309358,Aeromexico,Atlanta,RYY,Columbus,CMH,2019-03-13T13:50,2019-03-13T19:40,350,649.040868


### Asientos disponibles

In [45]:
info_aeropuertos['asientos_disponibles'] = info_aeropuertos.apply(lambda x: np.random.randint(0, 5), axis=1)
info_aeropuertos.sample(5)

,aerolinea,origen,cod_origen,destino,cod_destino,hora_salida,hora_llegada,duracion (min),precio,asientos_disponibles
130842,Aeromexico,Detroit,YIP,Atlanta,PDK,2019-03-12T08:45,2019-03-12T11:05,140,1096.941564,0
192193,Delta,Atlanta,FFC,Columbus,CSG,2019-03-15T11:35,2019-03-15T12:45,70,426.877911,1
27942,American,San Diego,SDM,Seattle,SEA,2019-03-07T20:43,2019-03-07T23:41,178,1089.893490,0
44570,Air Canada,Victoria,VCT,Vancouver,YVR,2019-03-07T06:49,2019-03-07T07:25,36,648.815516,1
358909,Air China,Hangzhou,HGH,Yuncheng,YCU,2019-03-12T06:01,2019-03-12T07:31,90,324.301110,2


### Últimas modificaciones

In [46]:
# Eliminamos columnas innecesarias
info_aeropuertos = info_aeropuertos.drop(columns=['cod_origen', 'cod_destino'])

In [47]:
# Pasamos a entero el precio (para mejorar la performance)
info_aeropuertos['precio'] = info_aeropuertos['precio'].astype(int)

In [56]:
# Renombramos la columna duracion
info_aeropuertos = info_aeropuertos.rename(index=str, columns={'duracion (min)':'duracion'})

In [63]:
# Eliminamos duraciones negativas
info_aeropuertos = info_aeropuertos.drop(info_aeropuertos[info_aeropuertos['duracion'] <= 0].index)
info_aeropuertos['duracion'].loc[info_aeropuertos['duracion'] <= 0].count()

0

In [64]:
info_aeropuertos.sample(5)

,aerolinea,origen,destino,hora_salida,hora_llegada,duracion,precio,asientos_disponibles
499385,Southwest WN 4668,Las Vegas,Atlanta,2019-03-23T09:10,2019-03-23T10:40,90,1280,1
302592,Air France,Santa Ana,Atlanta,2019-03-12T20:50,2019-03-12T22:55,125,446,3
137510,Alaska,Anchorage,Bethel,2019-03-10T06:30,2019-03-10T09:22,172,244,3
207242,Delta,Atlanta,Miami,2019-03-14T11:00,2019-03-14T12:10,70,367,0
189437,Delta,Atlanta,Columbia,2019-03-15T17:50,2019-03-16T19:55,1565,1000,2


In [65]:
info_aeropuertos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348664 entries, 2 to 525849
Data columns (total 8 columns):
aerolinea               348664 non-null object
origen                  348664 non-null object
destino                 348664 non-null object
hora_salida             348664 non-null object
hora_llegada            348664 non-null object
duracion                348664 non-null int64
precio                  348664 non-null int32
asientos_disponibles    348664 non-null int64
dtypes: int32(1), int64(2), object(5)
memory usage: 32.6+ MB


## Exportación del CSV

In [66]:
info_aeropuertos.to_csv('../csv/info_vuelos.csv', index = None, header = True)